In [46]:
import re
import json
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup as soup

# we don't really need matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [47]:
## these functions convert API json into friendly structure. 
## taken from http://stackoverflow.com/questions/956867/how-to-get-string-objects-instead-of-unicode-ones-from-json-in-python

def _decode_list(data):
    rv = []
    for item in data:
        if isinstance(item, unicode):
            item = item.encode('utf-8')
        elif isinstance(item, list):
            item = _decode_list(item)
        elif isinstance(item, dict):
            item = _decode_dict(item)
        rv.append(item)
    return rv

def _decode_dict(data):
    rv = {}
    for key, value in data.iteritems():
        if isinstance(key, unicode):
            key = key.encode('utf-8')
        if isinstance(value, unicode):
            value = value.encode('utf-8')
        elif isinstance(value, list):
            value = _decode_list(value)
        elif isinstance(value, dict):
            value = _decode_dict(value)
        rv[key] = value
    return rv

In [48]:
## RT API KEY
api_key = # key removed for shared version


In [49]:
def strip_dollarcomma(df, colset):
    ## strip $ signs and commas from scraped data
    nodollarcomma = re.compile('\\$|,')
    for cname in df.columns.values[colset]:
        df[cname] = [re.sub(nodollarcomma, '', str(val)) for val in df[cname]]
    return df

In [51]:
## scraping www.boxofficemojo.com for movie titles and box office revenue
#yr                  = 2010 #2010 #2011 #2012 #2013 #2014 #2015
yrs        = [2011,2012,2013,2014,2015,2016]
ct         = 0
cols       = ['title','year','tg','tg_adj','op','op_adj','tg_theaters','op_theaters','opdate','cldate','days','budget']
num_movies = 2500
num_cols   = len(cols)
init       = np.zeros((num_movies,num_cols))
df         = pd.DataFrame(data=init,columns=cols)    

for yr in yrs:
    url_page_numbers    = [1,2,3,4,5] if yr < 2016 else [1,2]
    
    print 'Collecting data for year: '+str(yr)
    
    ## open mojo URL, scrape everything, parse in beautiful soup
    for pnum in url_page_numbers:

        url      = 'http://www.boxofficemojo.com/yearly/chart/'
        opts     = {'page':pnum,'view':'releasedate','view2':'domestic','yr':yr,'p':'.htm'}
        query    = requests.get(url,params=opts).text.encode('utf8')
        soupdata = soup(query)

        ## these regex patterns are based on looking at the relevant HTML source code
        for cell in soupdata.find_all('td'):
            pattern_title = cell.find_all(href=re.compile('movies\\/\\?id'))
            pattern_tg = cell.find_all('b',text=re.compile('\\$\d'))
            pattern_tg_theaters = cell.find_all('b',text=re.compile('\\$\d'))

            ## save movie title
            if pattern_title:
                #print 'MOVIE'
                if len(list(cell.descendants)) < 5:

                    url2 = "http://www.boxofficemojo.com/"+cell.b.font.a['href']
                    try:
                        query2    = requests.get(url2).text.encode('utf8')
                    except Exception,e:
                        print "Failed on {}".format(url2)
                        pass
                    soupdata2 = soup(query2)
                    ## go into individual movie pages to get production budget info
                    ## note: this doesn't exist for all movies, especially low-budget films
                    for cell2 in soupdata2.find_all('td',valign='top'):#text=re.compile('Production\sBudget')):
                        pattern_budget = cell2.find_all(text=re.compile('Production Budget'))
                        if pattern_budget:
                            if cell2.b.text[0] == '$':
                                temp = cell2.b.text.split(" ")
                                if len(temp) > 1:
                                    if temp[1] == "million":
                                        scaleup = 1000000
                                    elif temp[1] == "thousand":
                                        scaleup = 1000
                                    budget = str(float(temp[0][1:]) * scaleup)
                                else:
                                    budget = temp[0]
                                df.ix[ct,'budget'] = budget
                                break
                df.ix[ct,'title'] = pattern_title[0].text

            ## save revenue, theater, and date info
            if ((len(pattern_tg)==1) and # sometimes this pattern brings back a ton of values
                (cell.next_sibling.font.text!='N/A') and
                (cell.next_sibling.next_sibling.font.text!='N/A') and
                (cell.next_sibling.next_sibling.next_sibling.font.text!='N/A')): 
                    #print 'TOTAL GROSS'
                    #print pattern_tg[0].text
                    df.ix[ct,'tg']          = pattern_tg[0].text
                    df.ix[ct,'tg_theaters'] = cell.next_sibling.font.text
                    df.ix[ct,'op']          = cell.next_sibling.next_sibling.font.text
                    df.ix[ct,'op_theaters'] = cell.next_sibling.next_sibling.next_sibling.font.text
                    df.ix[ct,'opdate']      = cell.next_sibling.next_sibling.next_sibling.next_sibling.font.text
                    df.ix[ct,'cldate']      = cell.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.font.text
                    df.ix[ct,'year']        = yr
                    
                    ct += 1    

In [52]:
mask = df.budget==0
df.ix[mask,'budget'] = np.nan

In [53]:
df = strip_dollarcomma(df, [2,4,6,7])

In [55]:
## find the last row of usable data and cut off the rest 
## (i made the dataframe bigger than it needs to be)

row_cutoff = False

for i,cl in enumerate(df.cldate):
    try:
        a = cl[0]
    except:
        row_cutoff = i-1 # number of available movies for a given year
        break

if not row_cutoff:
    row_cutoff = df.shape[0]+1
## create DATA_ variable (just to maintain variable naming convention)
DATA_mojo = df.ix[0:row_cutoff,:].copy() 

In [57]:
## add years to dates, compute number of days movie played in total
## if movie is still playing, compute up to current date
today = time.strftime('%m/%d/%Y') 

for i,cl in enumerate(DATA_mojo.cldate):
    #print DATA_mojo.title[i]
    op = DATA_mojo.opdate[i]
    yr = int(DATA_mojo.year[i])
 
    ## mojo sets the closing date as '-' for movies that are still playing
    ## this first part of the if/else is for movies that are NOT still playing
    if (str(cl[0])!='-'):
        try:
            opd = datetime.strptime(op, '%m/%d')
            cld = datetime.strptime(DATA_mojo.cldate[i], '%m/%d')
        except: # leap year bug - at first there are no years assigned so it goes back to 1970
            if op=='2/29':
                opd = datetime.strptime('2/28', '%m/%d')
                cld = datetime.strptime(DATA_mojo.cldate[i], '%m/%d')
            elif DATA_mojo.cldate[i]=='2/29':
                opd = datetime.strptime(op, '%m/%d')
                cld = datetime.strptime('2/28', '%m/%d')
        ## do the open and close dates cross over a year boundary?
        if (opd-cld).days > 0:
            DATA_mojo.cldate[i] += '/'+str(yr+1) 
            DATA_mojo.opdate[i] += '/'+str(yr) 
        else:
            DATA_mojo.cldate[i] += '/'+str(yr)
            DATA_mojo.opdate[i] += '/'+str(yr)
    ## this section deals with movies that ARE still playing
    else:
        temp = DATA_mojo.opdate[i]
        ## there is at least one 2015 movie which claims to have opened on a leap day! (which can't be.)
        if DATA_mojo.opdate[i][0:5] =='2/29':
            DATA_mojo.opdate[i] = '2/28'
        DATA_mojo.cldate[i] = today
        DATA_mojo.opdate[i] += '/'+str(int(yr))

    try:
        opd = datetime.strptime(DATA_mojo.opdate[i], '%m/%d/%Y')
    
        cld = datetime.strptime(DATA_mojo.cldate[i], '%m/%d/%Y')
        DATA_mojo.days[i] = abs((opd-cld).days)
    except Exception, e:
        print
        print 'problem!'
        print DATA_mojo.opdate[i]
        print str(e)
        break

/Users/andrew/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andrew/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andrew/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andrew/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be s

In [58]:
## create adjusted total and opening weekend gross revenue
## tg_adj = tg / (tg_theaters * days) = dollars per theater per day
## op_adj = op / (op_theaters)        = dollars per theater

DATA_mojo.op_adj = np.round(np.divide(DATA_mojo.op.values.astype(float),
                                      DATA_mojo.op_theaters.values.astype(float)),3)
DATA_mojo.tg_adj = np.round(np.divide(DATA_mojo.tg.values.astype(float), 
                                      DATA_mojo.tg_theaters.values.astype(float)*DATA_mojo.days.values.astype(float)),3)

DATA_mojo.head()

,title,year,tg,tg_adj,op,op_adj,tg_theaters,op_theaters,opdate,cldate,days,budget
0,Harry Potter and the Deathly Hallows Part 2,2011,381011219,659.760,169189427,38671.869,4375,4375,7/15/2011,11/24/2011,132,NaN
1,Transformers: Dark of the Moon,2011,352390543,813.219,97852865,23936.611,4088,4088,6/29/2011,10/13/2011,106,195000000.0
2,The Twilight Saga: Breaking Dawn Part 1,2011,281287133,713.199,138122261,34011.884,4066,4061,11/18/2011,2/23/2012,97,110000000.0
3,The Hangover Part II,2011,254464305,618.232,85946294,23774.908,3675,3615,5/26/2011,9/15/2011,112,80000000.0
4,Pirates of the Caribbean: On Stranger Tides,2011,241071802,438.593,90151958,21697.222,4164,4155,5/20/2011,9/29/2011,132,250000000.0


In [75]:
## sometimes (notably in 2010 data), movie titles will have non-ASCII chars (eg.'è')
## 'è' is the only exception i found from 2010-2015, so we replace it with 'e' here
## i checked RT and they do the same thing with their movie titles.
for i,t in enumerate(DATA_mojo.title):
    try:
        if re.search('è',t.encode('utf8')):
            DATA_mojo.title[i] = re.sub('è','e',t.encode('utf8'))
    except:
        pass

In [78]:
DATA_mojo.title = DATA_mojo.title.apply(lambda x : x.encode('utf-8'))

In [79]:
## write scraped mojo movie data to csv

DATA_mojo.to_csv("lda-data/mojo/mojoall.csv",index=False)

### This section is pre-processing before querying Rotten Tomatoes API

In [80]:
def strip_subtitles(df):
    ## strip movie strings of parenthetical titles and subtitles (ie. The Movie Title: the subtitle)
    pattern   = re.compile('\\(.+\\)')
    names = [re.sub(pattern,'',mov).encode('utf8') for mov in df.title]
    
    return names

In [81]:
DATA_mojo.title = strip_subtitles(DATA_mojo)

In [82]:
## replace whitespace with + for URI encoding
pattern      = re.compile('\s(\S)')
namesURI = [re.sub(pattern,'+\\1',mov).encode('utf8').strip().lower() for mov in DATA_mojo.title]

In [83]:
## replace colons with %3A URI encoding (not sure if this is necessary)
pattern = re.compile(':')
namesURI2 = [re.sub(pattern,'%3A',mov).encode('utf8') for mov in namesURI]

### Query RT review URLs using movie names from Mojo

In [84]:
DATA_mojo['revurl']       = None
DATA_mojo['critscore']    = None
DATA_mojo['critrate']     = None
DATA_mojo['audiscore']    = None
DATA_mojo['audirate']     = None
DATA_mojo['genres']       = None
DATA_mojo['id_rt']        = None
DATA_mojo['id_imdb']      = None
DATA_mojo['mpaa']         = None

In [85]:
def get_level2_param(x,level1,level2):
    try:
        imdb = x[level1][level2]
        return imdb
    except:
        return None

In [ ]:
## get RT review URLs for all movies from DATA_mojo
movielist = namesURI2
RESULTS   = np.empty(len(movielist),dtype="object")

for n,movie in enumerate(movielist[0:]):
    yr = DATA_mojo.year[n]
    if n%5==0:
        time.sleep(1) # RT only allows 5 API requests/second
     
    url             = 'http://api.rottentomatoes.com/api/public/v1.0/movies.json?q='+movie
    opts            = {'page_limit':4,'page':1,'apikey':api_key}
    query           = requests.get(url,params=opts).text
    try:
        DATA_           = json.loads(query, object_hook=_decode_dict)
    except:
        print query
        break
    # 'nomatch' is for reporting only - when we loop through results and don't find something
    # at first, if we eventually do find it we make note of this fact in the printout.
    nomatch = False 
    
    for x in DATA_['movies']:
        
        rt_yr       = x['year']
        rt_title    = x['title'].strip().lower()
        mj_title    = DATA_mojo.title[n].strip().lower()
        
        # sometimes the first hit is an old movie. also we need to strip and lowercase for matching.
        if ((rt_yr==yr)or(rt_yr==yr+1)) and (rt_title==mj_title) and (float(x['ratings']['critics_score'])>0) and (float(x['ratings']['audience_score'])>0):
            RESULTS[n]                = x['links']['reviews']
            DATA_mojo['revurl'][n]    = x['links']['reviews']
            DATA_mojo['critscore'][n] = x['ratings']['critics_score']
            DATA_mojo['critrate'][n]  = get_level2_param(x,'ratings','critics_rating')
            DATA_mojo['audiscore'][n] = x['ratings']['audience_score']
            DATA_mojo['audirate'][n]  = get_level2_param(x,'ratings','audience_rating')
        
            DATA_mojo['id_rt'][n]     = x['id']
            DATA_mojo['id_imdb'][n]   = get_level2_param(x,'alternate_ids','imdb')
            DATA_mojo['mpaa'][n]      = x['mpaa_rating']
            
            if nomatch:
                print 'MATCH FOUND: '+rt_title+' | YEAR: '+str(rt_yr)
            break
        else:
            nomatch = True
            print 'NO MATCH: TITLE: '+rt_title+' | YEAR: '+str(rt_yr)
            print mj_title+' != '+rt_title

/Users/andrew/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andrew/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andrew/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andrew/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be s

NO MATCH: TITLE: harry potter and the deathly hallows - part 2 | YEAR: 2011
harry potter and the deathly hallows part 2 != harry potter and the deathly hallows - part 2
NO MATCH: TITLE: twilight saga: breaking dawn part 1 | YEAR: 2011
the twilight saga: breaking dawn part 1 != twilight saga: breaking dawn part 1
NO MATCH: TITLE: the twilight saga: breaking dawn part 1 (extended edition) | YEAR: 2013
the twilight saga: breaking dawn part 1 != the twilight saga: breaking dawn part 1 (extended edition)
NO MATCH: TITLE: twilight saga double feature: eclipse and breaking dawn part 1 | YEAR: 2012
the twilight saga: breaking dawn part 1 != twilight saga double feature: eclipse and breaking dawn part 1
NO MATCH: TITLE: mission: impossible ghost protocol | YEAR: 2011
mission: impossible - ghost protocol != mission: impossible ghost protocol
NO MATCH: TITLE: i am thor | YEAR: 2015
thor != i am thor
MATCH FOUND: thor | YEAR: 2011
NO MATCH: TITLE: the great gilly hopkins | YEAR: 2016
hop != the gr

In [ ]:
## get rid of duplicate titles (there are only a few)
DATA_mojo.drop(DATA_mojo.index[np.where(DATA_mojo.duplicated(subset='title'))[0]],0,inplace=True)
DATA_mojo.reset_index(drop=True,inplace=True)

In [ ]:
DATA_mojo.head()

In [ ]:
## drop the movies we couldn't find reviews for
mask = []
for d in DATA_mojo['revurl']:
    mask.append((d is not None))
mojo2 = DATA_mojo[mask].copy().reset_index(drop=True)

In [ ]:
for n,rtid in enumerate(mojo2['id_rt']):
    if n%5==0:
        time.sleep(1) # RT only allows 5 API requests/second
   
    url   = 'http://api.rottentomatoes.com/api/public/v1.0/movies/{rtid}.json'.format(rtid=str(rtid))
    opts  = {'apikey':api_key}
    query = requests.get(url,params=opts).text
    DATA_ = json.loads(query, object_hook=_decode_dict)

    mojo2['genres'][n] = DATA_['genres'] 

### Pull first 20 RT review blurbs per movie

In [39]:
mojo2.revurl = mojo2.revurl.apply(lambda x: "http:"+x)

In [40]:
## take parsed URI-format movie titles and pull reviews from RT
N            = mojo2.shape[0]
K            = 20
DATA_reviews = np.empty((N,K),dtype='object')

for n,url in enumerate(mojo2.revurl):
    if n%100==0:
        print n       # report iteration number
    if n%5==0:
        time.sleep(1) # RT only allows 5 API requests/second
        
    opts  = {'apikey':api_key}
    query = requests.get(url, params=opts).text
    DATA_ = json.loads(query, object_hook=_decode_dict)

    for k,x in enumerate(DATA_['reviews']):
        DATA_reviews[n,k] = x['quote']


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500


In [41]:
## look for movies with no reviews, and exclude them from the final list
mask = []

DATA_reviews_cleaned = []
DATA_titles_cleaned  = []
for i,rev in enumerate(DATA_reviews):
    reviewed=False
    
    for r in rev:
        if not r==None:
            reviewed=True
    if not reviewed:
        print mojo2.title[i]+' has no review.'
        mask.append(False)
    else:
        DATA_reviews_cleaned.append(rev)
        DATA_titles_cleaned.append(mojo2.title[i])
        mask.append(True)

mojo3 = mojo2[mask].copy()

Believe has no review.
Busco Novio Para Mi Mujer has no review.


In [43]:
## write output to csv
np.savetxt("lda-data/reviews/reviewsall.csv", DATA_reviews_cleaned, fmt='%s', delimiter="__")
np.savetxt("lda-data/movietitles/titlesall.csv", DATA_titles_cleaned, fmt="%s", delimiter=",")

In [44]:
mojo3.to_csv("lda-data/mojo/mojoall.csv",index=False)